### Import

In [11]:
import os
import sys
import pandas as pd
import numpy as np
import shapefile
import re
import pickle
import networkx as nx
import matplotlib.pyplot as plt
from scipy.spatial import KDTree
from matplotlib.patches import Polygon as polygon
from matplotlib.collections import PatchCollection as patchCollection
from tqdm import tqdm
from collections import Counter
from scipy.spatial.distance import cdist
from scipy.spatial.distance import pdist

### Borough Codes

In [12]:
B = {}
B['manhattan'] = 1
B['thebronx'] = 2
B['brooklyn'] = 3
B['queens'] = 4
B['statenisland'] = 5

### Load Graph

In [13]:
X = pickle.load(open("../data/graphs/nyc.p","rb"))
O = X['O']
N = X['N']
edges = X['edges']
nodes = X['nodes']
nodenames = X['nodenames']
edgedatabase = X['edgedatabase']
G = X['G']
Nlist = X['Nlist']
pickle.dump(X,open('../data/graphs/nyc.p',"wb"))

### Read Chain Data

In [34]:
chaindatabase = pd.read_csv('../data/tables/chains_with_lat_lng_corrected.csv')
chainlatlng = chaindatabase[['lat','lng']].values
#np.unique(chaindatabase['city'].values)

### Convert Nodes to Coordinates

In [43]:
nodedatabase = []
for node in Nlist:
    elements = node.split(' | ')
    nodename = ' | '.join(elements[:-3])
    borough = elements[-1]
    if (borough == 'MANHATTAN'):
        borough = 'New York'
    if (borough == 'QUEENS'):
        borough = 'Queens'
    if (borough == 'THEBRONX'):
        borough = 'Bronx'
    if (borough == 'STATENISLAND'):
        borough = 'Staten Island'
    if (borough == 'BROOKLYN'):
        borough = 'Brooklyn'
    lat = float(elements[-2])
    lng = float(elements[-3])
    nodedatabase.append((node,borough,lat,lng))
nodedatabase = pd.DataFrame(nodedatabase)
nodedatabase.columns = ['nodename','borough','lat','lng']
nodelatlng = nodedatabase[['lat','lng']].values
nodedatabase.to_csv('../data/tables/nodes.csv',index=False)

### Create KDTree

In [44]:
kdtree = KDTree(nodelatlng)

### Find Node closest to chain lat lng

In [45]:
nodenames = []
count = 0
for i in range(chainlatlng.shape[0]):
    result = kdtree.query(chainlatlng[i,:],k=1)
    distance,indices = result
    chaindp = chaindatabase.iloc[i]
    nodedp = nodedatabase.iloc[indices]
    if (distance > 0.01):
        count = count + 1
        print('-------')
        print(distance)
        print(chaindp)
        print(nodedp)
        print(chaindp.city,nodedp.borough)
    else:
        nodenames.append(nodedp.nodename)
chaindatabase['nodename'] = nodenames

### Store

In [46]:
chaindatabase.to_csv('../data/tables/chains_with_nodes.csv',index=False)